In [121]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import date

def GetTnCashData(url):
    page = requests.get(url).text
    bsPage = BeautifulSoup(page)
    temp = bsPage.find_all("td",class_="SmallBlackText")
    winners = []
    prizes = []
    for i in range(1,8):
        winners.append(temp[3*i+1].get_text())
        prizes.append(temp[3*i+2].get_text().replace('$',''))
    return winners + prizes

def cleanDate(strdate):
    temp = strdate.split('/')
    return date(int(temp[2]),int(temp[0]),int(temp[1])).strftime('%Y-%m-%d')

tnc = open('tn_cash.csv','w')
tnc.write(','.join(['drawdate','n1','n2','n3','n4','n5','cashball','win51','win50','win41','win40','win31','win30','win21','prize51','prize50','prize41','prize40','prize31','prize30','prize21'])+'\n')
driver = webdriver.Firefox()
driver.get('https://www.tnlottery.com/winningnumbers/TennesseeCashlist.aspx?TCShowall=y#TennesseeCashball')
html = driver.page_source
nextLink = "navTennesseeCashNextPage"
soup = BeautifulSoup(html)
for pg in range(0,20):
    temp = soup.find_all("td",align="center")
    top = (len(temp)-4)/3 + 1
    for i in range(1,top):
        drawDate = [cleanDate(temp[3*i].get_text())]
        NumsDrawn = temp[3*i+1].get_text().replace('-',' ').split(' ')
        drawID = temp[3*i+2].a.get('href')
        drawID = drawID[drawID.index('=')+1:]
        drawID = drawID[:drawID.index("'")]
        drawData = GetTnCashData('https://www.tnlottery.com/winningnumbers/TennesseeCashdetails_popup.aspx?id='+drawID)
        tnc.write(','.join(drawDate + NumsDrawn + drawData) + '\n')
    driver.find_element_by_id(nextLink).click()
    soup = BeautifulSoup(driver.page_source)

tnc.close()
print 'Done'

Done
